In [ ]:
import sys; sys.path.append('..')
import sheet_convergence, sim_utils
from tri_mesh_viewer import TriMeshViewer
import numpy as np

In [ ]:
thickness = 0.05

In [ ]:
sheetResult = sheet_convergence.sheetConvergenceSweep(thickness, np.logspace(-1.5, -4, 15))
tetResult = sheet_convergence.tetConvergenceSweep(thickness, np.logspace(-3, -4, 15))
sheet_convergence.convergencePlot(thickness, sheetResult, tetResult)

In [ ]:
# Compare the midsurface strain distributions in the sheet and solid model.

In [ ]:
_, esheet = sheet_convergence.dirichletSheetSim(thickness, 0.0005)
_, esolid = sheet_convergence.dirichletTetSimulation(thickness, maxVol=5e-6)

In [ ]:
# Compute the vertex-averaged maximum principal strains
import field_sampler
fs = field_sampler.FieldSampler(esolid.mesh())

emax_sheet = [np.linalg.eigh(e)[0].max() for e in esheet.vertexGreenStrains()]
emax_solid = [np.linalg.eigh(e)[0].max() for e in esolid.vertexGreenStrains()]

# Also sample the tet simulation's piecewise linear vertex-averaged strain field at the midsurface sheet mesh vertices.
emax_solid_on_sheet = [np.linalg.eigh(e)[0].max() for e in fs.sample(esheet.mesh().vertices(),
                                                                     np.array(esolid.vertexGreenStrains()).reshape((-1, 9))).reshape(-1, 3, 3)]

In [ ]:
# Visualize strain and inspect mesh resolution on the undeformed configuration
v3 = TriMeshViewer(esolid.mesh(), scalarField=emax_solid, wireframe=True)
v3.show()

In [ ]:
import vis
from ipywidgets import HBox

vmin = min(np.min(emax_sheet), np.min(emax_solid_on_sheet))
vmax = max(np.max(emax_sheet), np.max(emax_solid_on_sheet))

sf_sheet = vis.fields.ScalarField(esheet, emax_sheet         , vmin=vmin, vmax=vmax)
sf_solid = vis.fields.ScalarField(esheet, emax_solid_on_sheet, vmin=vmin, vmax=vmax)

vdefo = TriMeshViewer(esolid, scalarField=emax_solid, wireframe=True) #volumetric deformation
sheetView = TriMeshViewer(esheet, scalarField=sf_sheet)
sampledSolidView = TriMeshViewer(esheet, scalarField=sf_solid)
HBox([sheetView.show(), sampledSolidView.show(), vdefo.show()])

In [ ]:
sampledSolidView.setCameraParams(sheetView.getCameraParams())
vdefo.setCameraParams(sheetView.getCameraParams())